In [ ]:
import pandas as pds
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CubicSpline, interp1d
from scipy.linalg import svd

In [ ]:
file_list = [
    "input celgene09.csv",
    "input centoco06.csv",
    "input cougar06.csv",
    "input novacea06.csv",
    "input pfizer08.csv",
    "input sanfi00.csv",
    "input sanofi79.csv",
    "inputS83OFF.csv",
    "inputS83ON.csv",
]

In [ ]:
df_list = list()
for f in file_list:
    df = pds.read_csv("data/" + f)
    df["name"] = df["name"].astype(str) + f
    df_list.append(df)

df = pds.concat(df_list)

In [ ]:
df.head()

In [ ]:
df_wide = pds.pivot_table(df, values="size", index="name", columns="date")

In [ ]:
df_wide.shape

In [ ]:
df_wide.head()

In [ ]:
(df_wide.isnull().sum(axis=0) < (3216 - 2)).values[:365]

In [ ]:
mat_wide = df_wide.values[:, :365].copy()
mat_wide.shape

In [ ]:
mat_wide.shape

In [ ]:
((365 - np.sum(np.isnan(mat_wide), axis=1)) > 10).sum()

In [ ]:
mat_small = mat_wide[((365 - np.sum(np.isnan(mat_wide), axis=1)) > 10)]

In [ ]:
mat_small.shape

In [ ]:
plt.plot(mat_small[0], "o")

In [ ]:
## scipy cubic spline interpolate

interp_list = []
for i in range(mat_small.shape[0]):

    row = mat_small[i]
    ind = np.where(~np.isnan(row))[0]
    val = row[~np.isnan(row)]

    #     cs = CubicSpline(ind, val, axis=0, bc_type='natural', extrapolate=None)
    cs = interp1d(ind, val, axis=0, kind="linear", fill_value="extrapolate")

    y_interp = cs(np.arange(mat_small.shape[1]))
    interp_list.append(y_interp)

In [ ]:
y_interp = np.stack(interp_list, axis=0)
y_interp.shape

In [ ]:
np.isnan(y_interp).sum()

In [ ]:
s, v, d = svd(y_interp)

In [ ]:
plt.plot(v[:5])

In [ ]:
plt.plot(d[0])

In [ ]:
d.shape

In [ ]:
# interpolate
# normalize
# cluster

y_interp_normalized = y_interp / y_interp[:, 0][:, None]

In [ ]:
inds = [
    0,
    1,
    2,
    9,
    10,
    12,
    13,
    16,
    20,
    23,
    24,
    31,
    35,
    39,
    41,
    42,
    52,
    54,
    59,
    61,
    63,
    65,
    67,
    69,
    72,
    73,
    75,
    78,
    81,
    82,
    83,
    84,
    87,
    88,
    91,
    92,
    93,
    95,
    99,
    101,
    103,
    104,
    106,
    108,
    112,
    114,
    115,
    116,
    117,
    125,
    126,
    129,
    130,
    132,
    133,
    134,
    138,
    139,
    141,
    143,
    144,
    145,
    146,
    147,
    148,
    150,
    154,
    156,
    157,
    159,
    162,
    165,
    166,
    167,
    168,
    170,
    175,
    176,
    178,
    179,
    182,
    187,
    190,
    191,
    192,
    199,
    202,
    204,
    211,
    213,
    214,
    216,
    217,
    218,
    219,
    223,
    224,
    225,
    226,
    227,
    228,
    229,
    230,
    233,
    235,
    239,
    241,
    244,
    245,
    247,
    250,
    252,
    254,
    256,
    257,
    258,
    260,
    262,
    264,
    265,
    266,
    268,
    269,
    270,
    272,
    275,
    276,
    277,
    278,
    279,
    280,
    281,
    282,
    284,
    286,
    289,
    290,
    291,
    295,
    297,
    299,
    301,
    303,
    305,
    306,
    307,
    308,
    312,
    315,
    316,
    317,
    321,
    322,
    323,
    325,
    326,
    327,
    329,
    331,
    333,
    334,
    340,
    346,
    347,
    348,
    351,
    352,
    357,
    359,
    365,
    366,
    367,
    368,
    369,
    370,
    371,
    372,
    374,
    376,
    377,
    378,
    379,
    380,
    383,
    384,
    385,
    386,
    387,
    388,
    389,
    390,
    393,
    394,
    395,
    396,
    397,
    399,
    401,
    405,
    407,
    410,
    412,
    413,
    417,
    421,
    422,
    425,
    427,
    430,
    431,
    432,
    435,
    439,
    441,
    442,
    445,
    446,
    447,
    450,
    452,
    454,
    455,
    456,
    457,
    458,
    459,
    460,
    461,
    462,
    463,
    465,
    466,
    469,
    470,
    473,
    474,
    475,
    479,
    484,
    485,
    486,
    488,
    489,
    490,
    491,
    492,
    494,
    496,
    497,
    500,
    502,
    503,
    504,
    505,
    509,
    510,
    512,
    513,
    514,
    515,
    516,
    517,
    518,
    519,
    520,
    523,
    524,
    525,
    526,
    528,
    529,
    531,
    534,
    535,
    536,
    537,
    539,
    540,
    541,
    542,
    543,
    544,
    545,
    546,
    547,
    550,
    552,
    553,
    555,
    556,
    557,
    559,
    561,
    563,
    564,
    568,
    569,
    570,
    576,
    578,
    580,
    581,
    582,
    583,
    585,
    586,
    588,
    593,
    594,
    597,
]

In [ ]:
y_c1 = y_interp_normalized[inds]

In [ ]:
plt.plot(y_c1[0].T)

In [ ]:
y_c1[0].shape

In [ ]:
s, v, d = svd(y_c1)

In [ ]:
plt.plot(v)

In [ ]:
def build(y_interp, ind):
    y_time = np.arange(y_interp.shape[1]) / 365

    y_time_list = [y_time] * y_interp.shape[0]

    y_time = np.stack(y_time_list, axis=0).T[:, :, None]

    y_interp = y_interp.T[:, :, None]

    y_total = np.concatenate([y_interp, y_time], axis=-1)

    mask = ~np.isnan(mat_small[inds]).T
    return y_total, mask

In [ ]:
y_total_c1, mask_c1 = build(y_c1, 0)

In [ ]:
y_total_c1.shape

In [ ]:
mask_c1.shape

In [ ]:
y_total_c1[:, :, 0].std()

In [ ]:
y_total_c1[:, :, 1].std()

In [ ]:
import pickle

with open("data/real_data_c1.pkl", "wb") as f:
    pickle.dump(y_total_c1, f)

with open("data/real_data_mask_c1.pkl", "wb") as f:
    pickle.dump(mask_c1, f)